In [39]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from collections import deque

In [149]:
def calculateTrajectory(incAngle, speed, mass, charge, df_resolution=5, step_size=np.float64(0.01), max_iters=4000000):
    """
    Calculate the trajectory of the particle without taking any time retardation into account.
    
    Args:
        incAngle (np.float64): Angle between particle's velocity and superconductor's plane, [radians].
        speed (np.float64): Velocity of the particle, [c].
        mass (np.float64): Mass of the particle, [amu].
        charge (np.float64): Charge of the particle, [e].
        df_resolution (int): Resolution of the trajectory being recorded. Smaller is better.
        step_size(np.float64): Step size of the solver, typical values are in [0.01, 0.1]. Smaller is better.
        max_iters (int): Maximum number of iterations of the solver. 
    
    Returns:
        np.float64: distance of closest approach to the superconductor, [meters].
        
        pd.dataFrame: Trajectory of the particle near superconductor, [{pX}, {pY}, {uX}, {uY}, {aX}, {aY}].
        
        [float, float]: Closest position of the particle to the superconductor, [h_min].
    """ 
    def dUdt(uX_, uY_, xY_):
        xY_2 = np.square(xY_)
        return (-K * uX_ * uY_ / xY_2), (K * uX_ * uX_ / xY_2)
    
    uX = np.float64(speed * np.cos(incAngle))
    uY = np.float64(-speed * np.sin(incAngle))
    pX = -uX * np.float64(10000)
    pY = -uY * np.float64(10000)
    edgeY = pY
    gamma = 1 / np.sqrt(1 - np.square(speed))
    
    h_min = np.float64(-1.5455e-18) * (np.square(charge) / (mass * np.log(np.cos(incAngle))))
    K = np.float64(-np.log(np.cos(incAngle)) / gamma)
    
    df = pd.DataFrame(columns=['pX', 'pY', 'uX', 'uY', 'aX', 'aY'])
    tick = 0
    lowestX, lowestY = pX, pY
    half_step = np.float64(step_size / 2)
    while pY <= edgeY and tick < max_iters:
        dUxdt, dUydt = dUdt(uX, uY, pY) # acceleration at current position for estimating values at t + dt/2
        dUxdt, dUydt = dUdt(uX + (dUxdt * half_step), uY + (dUydt * half_step), pY + (uY * half_step)) #midpoint
        
        uX += dUxdt * step_size
        uY += dUydt * step_size
        pX += uX * step_size
        pY += uY * step_size
        
        if pY < lowestY: # record the point of closest approach to the superconductor
            lowestY = pY
            lowestX = pX
        if pY < 3 and tick % df_resolution == 0: # record points into dataframe if they are sufficiently close to superconductor
            df.loc[len(df)] = {'pX': pX, 'pY': pY, 'uX': uX, 'uY': uY, 'aX': dUxdt, 'aY': dUydt}
        tick += 1
    
    return h_min * lowestY, df, [float(lowestX), float(lowestY)]


def calculateTrajectory_timeRetardation(incAngle, speed, mass, charge, df_resolution=5, step_size=np.float64(0.01), max_iters=4000000):
    """
    Calculate the trajectory of the particle, taking time retardation into account.
    
    Args:
        incAngle (np.float64): Angle between particle's velocity and superconductor's plane, [radians].
        speed (np.float64): Velocity of the particle, [c].
        mass (np.float64): Mass of the particle, [amu].
        charge (np.float64): Charge of the particle, [e].
        df_resolution (int): Resolution of the trajectory being recorded. Smaller is better.
        step_size(np.float64): Step size of the solver, typical values are in [0.01, 0.1]. Smaller is better.
        max_iters (int): Maximum number of iterations of the solver. 
    
    Returns:
        np.float64: distance of closest approach to the superconductor, [meters].
        
        pd.dataFrame: Trajectory of the particle near superconductor, [{pX}, {pY}, {uX}, {uY}, {aX}, {aY}].
        
        [float, float]: Closest position of the particle to the superconductor, [h_min].
    """ 
    def dUdt(uX_, uY_, xY_):
        xY_2 = np.square(xY_)
        return (-K * uX_ * uY_ / xY_2), (K * uX_ * uX_ / xY_2)
    def dUdt_del(uX1, uY1, uX0, uY0, pX1, pY1, pX0, pY0):
        pathX = pX1 - pX0
        pathY = pY1 + pY0
        path2 = np.square(pathX) + np.square(pathY)
        sine = np.sqrt(1 - (np.square(pathX*uX0 - pathY*uY0) / (path2 *(np.square(uX0) + np.square(uY0))))) # from 1 - cos^2
        return -G * sine * uY1 / path2, G * sine * uX1 / path2
    
    uX = np.float64(speed * np.cos(incAngle))
    uY = np.float64(-speed * np.sin(incAngle))
    pX = -uX * np.float64(5000)
    pY = -uY * np.float64(5000)
    edgeY = pY
    gamma = 1 / np.sqrt(1 - np.square(speed))
    
    h_min = np.float64(-1.5455e-18) * (np.square(charge) / (mass * np.log(np.cos(incAngle))))
    K = np.float64(-np.log(np.cos(incAngle)) / gamma)
    G = speed * K
    
    df = pd.DataFrame(columns=['pX', 'pY', 'uX', 'uY', 'aX', 'aY'])
    tick = 0
    lowestX, lowestY = pX, pY
    half_step = np.float64(step_size / 2)
    history = deque()
    while pY <= edgeY and tick < max_iters:
        deltaT = (2 * pY / step_size).astype(int)
        if tick > deltaT:
            uX_prev, uY_prev, pX_prev, pY_prev = history[-deltaT]
            
            dUxdt, dUydt = dUdt_del(uX, uY, uX_prev, uY_prev, pX, pY, pX_prev, pY_prev)
            dUxdt, dUydt = dUdt_del(uX + (dUxdt * half_step), uY + (dUydt * half_step), uX_prev, uY_prev, 
                                    pX + (uX * half_step), pY + (uY * half_step), pX_prev, pY_prev)
            
            uX += dUxdt * step_size
            uY += dUydt * step_size
            pX += uX * step_size
            pY += uY * step_size
            history.append([uX, uY, pX, pY])

            if pY < lowestY: # record the point of closest approach to the superconductor
                lowestY = pY
                lowestX = pX
            if pY < 3 and tick % df_resolution == 0: # record points into dataframe if they are sufficiently close to superconductor
                df.loc[len(df)] = {'pX': pX, 'pY': pY, 'uX': uX, 'uY': uY, 'aX': dUxdt, 'aY': dUydt}
        else:
            dUxdt, dUydt = dUdt(uX, uY, pY)
            dUxdt, dUydt = dUdt(uX + (dUxdt * half_step), uY + (dUydt * half_step), pY + (uY * half_step))
            uX += dUxdt * step_size
            uY += dUydt * step_size
            pX += uX * step_size
            pY += uY * step_size
            history.append([uX, uY, pX, pY])
        tick += 1
    
    return h_min * lowestY, df, [float(lowestX), float(lowestY)]


def drawPlot(h_min, df, low_pos):
    fig = go.Figure(data=go.Scatter(x=df['pX'].astype(float), y=df['pY'].astype(float), mode='markers', name='Positions', marker=dict(size=3, color='#ea5545')))
    line_x = []
    line_y = []
    # Fill these lists
    for i in range(len(df)):
        line_x.extend([df.loc[i, 'pX'].astype(float), (df.loc[i, 'pX'] + (0.1 * df.loc[i, 'aX']/abs(df.loc[i, 'aX']))).astype(float), None]) # None to separate segments
        line_y.extend([df.loc[i, 'pY'].astype(float), (df.loc[i, 'pY'] + (0.1 * df.loc[i, 'aY']/abs(df.loc[i, 'aY']))).astype(float), None]) # None to separate segments
    
    # Add a single trace for all acceleration vectors
    fig.add_trace(
        go.Scatter(
            x=line_x,
            y=line_y,
            mode='lines',
            line=dict(width=1, color='#a4a2a8'),
            name='Acceleration Vectors'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[float(df['pX'].min()), float(df['pX'].max())], 
            y=[0, 0],
            mode='lines',
            line=dict(width=1, color='white'), 
            name='SuperConductor'))
    
    fig.add_trace(
        go.Scatter(
            x=[low_pos[0], low_pos[0]], 
            y=[low_pos[1] * 0.1, low_pos[1] * 0.9],
            mode='lines+markers',
            line=dict(width=2, color='#edbf33'),
            marker=dict(symbol="x",size=4, color='#edbf33'),
            name='Approach Distance'))
    
    fig.add_annotation(
        x=float(low_pos[0]), 
        y=-0.2,
        text="Approach Distance: " + str(np.round(low_pos[1], 3)) + " of h_min (non-relativistic),<br>or " + str(np.round(h_min * 10e18, 3)) + " attometers",
        showarrow=False,
            font=dict(
            size=16,  # Font size
            color='white'  # Font color
        ),
    )
    # Update layouts
    fig.update_layout(
        title='Particle Movement',
        title_font=dict(size=24), # Font size for legend title
        xaxis_title='X Position',
        yaxis_title='Y Position',
        xaxis=dict(showgrid=True, zeroline=False),
        yaxis=dict(showgrid=True, zeroline=False),
        xaxis_title_font=dict(size=20),  # Font size for x-axis title
        yaxis_title_font=dict(size=20),  # Font size for y-axis title
        legend_title_font=dict(size=18), # Font size for legend title
        legend=dict(font=dict(size=18)), # Font size for legend text
        legend_title_text='Trace Types'
    )
    # Update font size for x-axis and y-axis tick labels
    fig.update_xaxes(tickfont=dict(size=16))  # Customize x-axis tick font
    fig.update_yaxes(tickfont=dict(size=16))  # Customize y-axis tick font
    return fig

In [147]:
incidenceAngle = np.float64(np.pi / 4) # angle of particle's movement to superconductor's plane, in radians
particleSpeed = np.float64(0.8) # particle's speed in c
particleMass = np.float64(1.007316) # particle's mass in amu
particleCharge = np.float64(1) # particle's charge in e

In [150]:
drawPlot(*calculateTrajectory(incidenceAngle, particleSpeed, particleMass, particleCharge)).write_html('plots/p_45_08_nonret.html')
drawPlot(*calculateTrajectory_timeRetardation(incidenceAngle, particleSpeed, particleMass, particleCharge)).write_html('plots/p_45_08_ret.html')

In [44]:
incidenceAngle_hmin_df_test = pd.DataFrame(columns=['particle', 'angle', 'hmin'])
particleSpeed = 0.8 # particle's speed in c
particleMass = np.float64(1.007316) # particle's mass in amu
particleCharge = np.float64(1) # particle's charge in e
for i in range(1, 12):
    h_min, _, point = calculateTrajectory(np.float64(i * np.pi/24), particleSpeed, particleMass, particleCharge)
    incidenceAngle_hmin_df_test.loc[len(incidenceAngle_hmin_df_test)] = {'particle': "proton (1252 MeV/c)",'angle': np.float64(i * np.pi/24), 'hmin': h_min * np.float64(10e18)}

In [67]:
incidenceAngle_hmin_df = pd.DataFrame(columns=['particle', 'angle', 'hmin'])
particleSpeed = 0.8 # particle's speed in c
particleMass = np.float64(1.007316) # particle's mass in amu
particleCharge = np.float64(1) # particle's charge in e
for i in range(2, 18):
    h_min, _, point = calculateTrajectory(np.float64(i * np.pi/36), particleSpeed, particleMass, particleCharge)
    incidenceAngle_hmin_df.loc[len(incidenceAngle_hmin_df)] = {'particle': "proton (1252 MeV/c)",'angle': i * 5, 'hmin': h_min * np.float64(10e18)}

particleSpeed = 0.95 # particle's speed in c
particleMass = np.float64(1.007316) # particle's mass in amu
particleCharge = np.float64(1) # particle's charge in e
for i in range(2, 18):
    h_min, _, point = calculateTrajectory(np.float64(i * np.pi/36), particleSpeed, particleMass, particleCharge)
    incidenceAngle_hmin_df.loc[len(incidenceAngle_hmin_df)] = {'particle': "proton (2859 MeV/c)", 'angle': i * 5, 'hmin': h_min * np.float64(10e18)}
    
particleSpeed = 0.7 # particle's speed in c
particleMass = np.float64(4.001506) # particle's mass in amu
particleCharge = np.float64(2) # particle's charge in e
for i in range(2, 18):
    h_min, _, point = calculateTrajectory(np.float64(i * np.pi/36), particleSpeed, particleMass, particleCharge)
    incidenceAngle_hmin_df.loc[len(incidenceAngle_hmin_df)] = {'particle': "alpha (3659 MeV/c)", 'angle': i * 5, 'hmin': h_min * np.float64(10e18)}

In [79]:
colors = {'proton (1252 MeV/c)': '#edbf33', 'alpha (3659 MeV/c)': '#ea5545', 'proton (2859 MeV/c)': '#87bc45'}
fig = px.line(x=incidenceAngle_hmin_df['angle'].astype(float), y=incidenceAngle_hmin_df['hmin'].astype(float), color=incidenceAngle_hmin_df['particle'], color_discrete_map=colors, markers=True)
# fig = go.Figure(data=go.Scatter(x=incidenceAngle_hmin_df['angle'].astype(float), y=incidenceAngle_hmin_df['hmin'].astype(float), mode='line+markers', name=incidenceAngle_hmin_df['particle'], marker=dict(size=2, color=incidenceAngle_hmin_df['particle'].map(colors)), line=dict(width=2, color=incidenceAngle_hmin_df['particle'].map(colors))))
fig.update_traces(marker=dict(size=6))
fig.update_layout(
    title='Approach Distance for Cosmic Ray Particles',
    title_font=dict(size=24), # Font size for legend title
    xaxis_title='Angle of Incidence, [deg]',
    yaxis_title='Closest Approach, [atto m]',
    xaxis=dict(showgrid=True, zeroline=False),
    yaxis=dict(showgrid=True, zeroline=False),
    xaxis_title_font=dict(size=20),  # Font size for x-axis title
    yaxis_title_font=dict(size=20),  # Font size for y-axis title
    legend_title_font=dict(size=18), # Font size for legend title
    legend=dict(font=dict(size=18)), # Font size for legend text
    legend_title_text='Particles'
)
# Update font size for x-axis and y-axis tick labels
fig.update_xaxes(tickfont=dict(size=16))  # Customize x-axis tick font
fig.update_yaxes(tickfont=dict(size=16))  # Customize y-axis tick font

fig.write_html('plots/hmin_cosmicrayparticles.html')

In [14]:
def dUdt(K, uX_norm, uY_norm, xY_norm):
    xY_norm2 = np.square(xY_norm)
    return (-K * uX_norm * uY_norm / xY_norm2), (K * uX_norm * uX_norm / xY_norm2)
def dUdt_del(G, uX_norm1, uY_norm1, uX_norm0, uY_norm0, xX_norm1, xY_norm1, xX_norm0, xY_norm0):
    pathX = xX_norm1 - xX_norm0
    pathY = xY_norm1 + xY_norm0
    path2 = np.square(pathX) + np.square(pathY)
    sine = np.sqrt(1 - (np.square(pathX*uX_norm0 - pathY*uY_norm0) / (path2 + np.square(uX_norm0) + np.square(uY_norm0))))
    return -G * sine * uY_norm1 / path2, G * sine * uX_norm1 / path2

uX_norm = np.float128(0.5656)
uY_norm = np.float128(-0.5656)

charge = np.float128(1.602176634e-19)
m0 = np.float128(1.6726e-27)

sp_mod = np.sqrt(np.square(uX_norm) + np.square(uY_norm)) * np.float128(299792458)
minApproach_nr = -np.float128(1e-7) * np.square(charge) / (4 * m0 * np.log(uX_norm * np.float128(299792458) / sp_mod))
# minApproach_nr = np.float128(1e-9)
gamma = 1 / np.sqrt(1 - (np.square(uX_norm) + np.square(uY_norm)))

# xX_norm = np.float128(-1e-12) / minApproach_nr
# xY_norm = np.float128(1e-12) / minApproach_nr
xX_norm = -5000
xY_norm = 5000
edgeX_norm = -xX_norm
edgeY_norm = xY_norm

K = np.float128(1e-7) * np.square(charge)  / (4 * minApproach_nr * m0 * gamma)
G = K * np.sqrt(np.square(uX_norm) + np.square(uY_norm))

display ("speed: " + str(sp_mod) + " m/s")
display ("momentum: " + str(1.871e21 * m0 * sp_mod / np.sqrt(1 - np.square(sp_mod / np.float128(299792458)))) + " MeV/c")
display ("approach distance: " + str(minApproach_nr) + " m")
df = pd.DataFrame(columns=['pX', 'pY', 'uX', 'uY', 'aX', 'aY'])

'speed: 239797748.73643351627 m/s'

'momentum: 1250.3833397429233742 MeV/c'

'approach distance: 1.1070652997823073345e-18 m'

In [15]:
-np.log(np.cos(np.pi / 4)) / gamma

0.20799996813045583067

In [16]:
# Without taking information delay into account
tick = 0
lowestY = xY_norm
highestuY = uY_norm
resolution = 3 # lower number means higher resolution of dataframe
while xX_norm < edgeX_norm and tick <= 30000000:
    dt = 0.01 #step size, typically 0.01 - 0.1 are enough
    
    dUxdt, dUydt = dUdt(K, uX_norm, uY_norm, xY_norm) # acceleration at current position for estimating values at t + dt/2
    dUxdt, dUydt = dUdt(K, uX_norm + (dUxdt * dt * 0.5), uY_norm + (dUydt * dt * 0.5), xY_norm + (uY_norm * dt * 0.5)) #midpoint
    
    uX_norm += dUxdt * dt
    uY_norm += dUydt * dt
    xX_norm += uX_norm * dt
    xY_norm += uY_norm * dt
    
    if xY_norm < lowestY: # record the point of closest approach to the superconductor
        lowestY = xY_norm
        lowestX = xX_norm
    if xY_norm < 3 and tick % resolution == 0: # record points into dataframe if they are sufficiently close to superconductor
        df.loc[len(df)] = {'pX': xX_norm, 'pY': xY_norm, 'uX': uX_norm, 'uY': uY_norm, 'aX': dUxdt, 'aY': dUydt}
    tick += 1

In [ ]:
# Taking information delay into account
tick = 0
lowestY = xY_norm
lowestX = 0
highestuY = uY_norm
history = deque()
resolution = 3 # lower number means higher resolution of dataframe
while xY_norm <= edgeY_norm and tick <= 2000000:
    dt = 0.01
    if tick > 2 * xY_norm / dt:
        uX_norm_prev, uY_norm_prev, xX_norm_prev, xY_norm_prev = history[-(2 * xY_norm / dt).astype(int)]
        
        dUxdt, dUydt = dUdt_del(G, uX_norm, uY_norm, uX_norm_prev, uY_norm_prev, xX_norm, xY_norm, xX_norm_prev, xY_norm_prev)
        dUxdt, dUydt = dUdt_del(G, uX_norm + (dUxdt * dt * 0.5), uY_norm + (dUydt * dt * 0.5), uX_norm_prev, uY_norm_prev, xX_norm + (uX_norm * dt * 0.5), xY_norm + (uY_norm * dt * 0.5), xX_norm_prev, xY_norm_prev)
        
        uX_norm += dUxdt * dt
        uY_norm += dUydt * dt
        xX_norm += uX_norm * dt
        xY_norm += uY_norm * dt
        history.append([uX_norm, uY_norm, xX_norm, xY_norm])
        # history.popleft()
        if xY_norm < lowestY:
            lowestY = xY_norm
            lowestX = xX_norm
        if xY_norm < 3 and tick % resolution == 0:
            df.loc[len(df)] = {'pX': xX_norm, 'pY': xY_norm, 'aX': dUxdt, 'aY': dUydt}
    else:
        dUxdt, dUydt = dUdt(K, uX_norm, uY_norm, xY_norm)
        dUxdt, dUydt = dUdt(K, uX_norm + (dUxdt * dt * 0.5), uY_norm + (dUydt * dt * 0.5), xY_norm + (uY_norm * dt * 0.5))
        uX_norm += dUxdt * dt
        uY_norm += dUydt * dt
        xX_norm += uX_norm * dt
        xY_norm += uY_norm * dt
        history.append([uX_norm, uY_norm, xX_norm, xY_norm])
    tick += 1

In [23]:
# Create a scatter plot for positions
fig = go.Figure(data=go.Scatter(x=df['pX'].astype(float), y=df['pY'].astype(float), mode='markers', name='Positions', marker=dict(size=2, color='red')))
line_x = []
line_y = []
# Fill these lists
for i in range(len(df)):
    line_x.extend([df.loc[i, 'pX'].astype(float), (df.loc[i, 'pX'] + (0.1 * df.loc[i, 'aX']/abs(df.loc[i, 'aX']))).astype(float), None]) # None to separate segments
    line_y.extend([df.loc[i, 'pY'].astype(float), (df.loc[i, 'pY'] + (0.1 * df.loc[i, 'aY']/abs(df.loc[i, 'aY']))).astype(float), None]) # None to separate segments

# Add a single trace for all acceleration vectors
fig.add_trace(
    go.Scatter(
        x=line_x,
        y=line_y,
        mode='lines',
        line=dict(width=1, color='yellow'),
        name='Acceleration Vectors'
    )
)
fig.add_trace(
    go.Scatter(
        x=[float(df['pX'].min()), float(df['pX'].max())], 
        y=[0, 0],
        mode='lines',
        line=dict(width=3, color='white'), 
        name='SuperConductor'))

fig.add_trace(
    go.Scatter(
        x=[float(lowestX), float(lowestX)], 
        y=[0, float(lowestY)],
        mode='lines+markers',
        line=dict(width=2, color='green'),
        marker=dict(size=4, color='green'),
        name='Approach Distance'))
fig.add_annotation(
    x=0, y=0.3,
    text="Approach Distance: " + str(np.round(lowestY, 3)) + " of h_min",
    showarrow=False,
        font=dict(
        size=14,  # Font size
        color='white'  # Font color
    )
)
# Update layouts
fig.update_layout(
    title='Particle Movement',
    xaxis_title='X Position',
    yaxis_title='Y Position',
    xaxis=dict(showgrid=True, zeroline=False),
    yaxis=dict(showgrid=True, zeroline=False),
    legend_title_text='Trace Types'
)

# Show the plot
fig.write_html('plot_delay.html')